In [0]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.covariance import EmpiricalCovariance
from scipy.stats import multivariate_normal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

Read data from csv file

In [2]:
data = pd.read_csv("phenome.csv")

df = pd.DataFrame(data)

split_ratio = 0.2
test_size = int(split_ratio * len(data))
print(test_size)
df

901


,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,x.11,x.12,x.13,x.14,x.15,x.16,x.17,x.18,x.19,x.20,x.21,x.22,x.23,x.24,x.25,x.26,x.27,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,x.37,x.38,x.39,x.40,...,x.218,x.219,x.220,x.221,x.222,x.223,x.224,x.225,x.226,x.227,x.228,x.229,x.230,x.231,x.232,x.233,x.234,x.235,x.236,x.237,x.238,x.239,x.240,x.241,x.242,x.243,x.244,x.245,x.246,x.247,x.248,x.249,x.250,x.251,x.252,x.253,x.254,x.255,x.256,label
0,9.85770,9.20711,9.81689,9.01692,9.05675,8.92518,11.28308,11.52980,10.79713,9.04747,11.38065,11.61545,10.31166,12.14300,10.65254,11.19395,11.27878,11.13465,11.82706,12.81048,12.32239,10.27542,9.57760,10.62308,11.34983,11.58342,12.19465,10.67933,9.95300,11.47047,11.74805,11.28167,11.51648,12.50765,12.10918,11.97849,10.80743,9.65682,10.71985,9.50841,...,12.29207,12.57110,10.00457,12.09898,14.08718,13.97254,11.74888,13.23555,13.76750,13.53994,12.28767,13.48164,13.43467,11.43232,11.01716,13.02529,13.37422,7.72797,12.32757,13.37041,12.68903,13.41566,13.16755,13.62844,13.32554,12.46720,13.84955,10.83688,11.23451,13.19359,12.94519,12.68076,11.20767,13.69394,13.72055,12.16628,12.92489,12.51195,9.75527,4
1,13.23079,14.19189,15.34428,18.11737,19.53875,18.32726,17.34169,17.16861,19.63557,20.15212,18.84739,19.35436,19.61419,20.32628,21.41752,20.14227,17.97866,16.32943,17.63076,16.51955,15.95348,16.77282,16.98819,18.32396,17.59234,15.94519,13.31152,15.36856,16.10275,14.76571,14.85925,11.89751,14.90796,15.49864,15.13160,14.41750,11.42785,14.15725,13.80756,14.41612,...,12.52542,11.56420,10.72840,11.34892,11.10939,9.53083,11.11848,11.27136,11.29874,10.02574,10.11055,7.53453,8.71152,9.58630,9.71517,9.29711,7.74722,10.07586,6.80730,7.99336,8.70087,6.50054,9.11663,4.95653,8.98125,7.78602,8.06256,7.46077,7.03316,7.93572,8.91567,8.45714,8.77266,9.59717,8.45336,7.57730,5.38504,9.43063,8.59328,3
2,10.81889,9.07615,9.77940,12.20135,12.59005,10.53364,8.54693,9.46049,11.96755,12.05282,8.92333,7.23825,6.98416,7.50434,7.21911,6.10589,5.37987,5.69972,6.15451,8.33365,8.61714,6.58075,5.63212,4.57192,7.18243,7.64013,6.48067,6.51159,4.77806,6.32551,5.61790,5.93619,3.93011,5.45621,5.22023,5.68300,6.26205,7.25231,6.78937,5.98700,...,3.16328,4.25020,7.19742,7.27264,5.77017,5.66405,6.48652,5.99069,2.37876,5.10594,5.88512,5.65892,6.57863,4.45805,4.86107,5.52299,6.17580,6.20610,6.49488,6.79143,5.87053,6.65905,6.37389,4.16703,5.47719,5.43269,4.10159,3.91116,3.88583,5.83309,6.49345,5.00824,5.51019,5.95725,7.04992,7.02469,6.58416,6.27058,3.85042,2
3,10.53679,9.12147,10.84621,13.92331,13.52476,10.27831,8.97459,11.57109,12.35839,10.47826,8.56676,8.97603,10.67954,9.67530,8.83348,8.43713,5.42489,6.46217,7.50640,3.71255,5.43962,7.34509,7.94968,7.08204,3.99006,4.51822,4.95940,4.61728,2.99881,4.09136,7.16309,6.73208,6.09821,6.09049,5.35111,7.51708,7.34681,6.55168,7.69089,8.40109,...,4.80601,6.87162,6.98889,5.83639,4.38529,5.09759,1.45570,4.44739,5.95291,5.73163,3.45180,4.49433,4.79405,5.97959,3.41728,5.65843,6.55074,5.15614,4.18213,4.95448,5.61000,5.15144,5.44564,4.74798,5.55902,2.88668,4.56161,5.62022,5.51356,5.95977,5.36506,5.85688,5.40324,6.07126,5.30651,4.27412,3.63384,3.22823,4.63123,2
4,12.96705,13.69454,14.91182,18.22292,18.45390,17.25760,17.79614,17.76387,18.99632,17.40394,18.08701,17.78545,18.58141,18.26168,18.72958,19.56382,18.81372,19.77800,18.92736,21.83878,21.83288,20.81927,21.14635,17.96529,21.40177,20.30099,19.61594,19.58996,19.47810,17.59408,17.13690,18.10193,17.84130,18.00640,17.14704,17.84642,18.23249,18.65123,17.67551,18.56012,...,7.34476,6.45842,5.64810,8.47157,4.47899,8.38229,8.23281,8.35911,8.45261,8.27508,7.96965,9.42980,8.83366,6.59190,8.20029,8.89709,9.71158,9.85747,8.77021,9.17960,10.49989,9.99960,8.33319,9.29810,4.60656,7.58772,7.82380,6.96809,6.97810,8.91020,7.74256,8.00151,7.58624,6.65202,7.69109,6.93683,7.03600,7.01278,8.52197,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [3]:
df = shuffle(df)
df

,x.1,x.2,x.3,x.4,x.5,x.6,x.7,x.8,x.9,x.10,x.11,x.12,x.13,x.14,x.15,x.16,x.17,x.18,x.19,x.20,x.21,x.22,x.23,x.24,x.25,x.26,x.27,x.28,x.29,x.30,x.31,x.32,x.33,x.34,x.35,x.36,x.37,x.38,x.39,x.40,...,x.218,x.219,x.220,x.221,x.222,x.223,x.224,x.225,x.226,x.227,x.228,x.229,x.230,x.231,x.232,x.233,x.234,x.235,x.236,x.237,x.238,x.239,x.240,x.241,x.242,x.243,x.244,x.245,x.246,x.247,x.248,x.249,x.250,x.251,x.252,x.253,x.254,x.255,x.256,label
823,9.78745,12.09806,17.05684,17.75944,14.32867,16.34835,19.79358,19.61493,16.29265,19.27424,21.23854,20.02514,17.48206,17.50251,18.90506,16.78379,15.10436,17.18835,17.12597,14.40361,14.72934,16.26761,15.69398,14.07179,13.37151,15.47702,14.20683,13.14875,14.08007,14.88258,12.36411,13.30274,14.48868,13.97309,12.66073,12.39523,14.12373,12.66214,12.97868,13.33853,...,7.25774,8.24811,7.00327,7.26950,7.40647,7.69632,7.39751,8.13679,7.56746,7.63304,6.11913,7.70188,8.31495,8.14269,3.86973,6.75918,7.28548,8.05626,5.94538,7.80349,7.41342,7.45088,8.31350,8.47600,8.75193,8.96346,8.28764,10.59140,10.80520,11.37802,8.19421,10.25559,11.21173,10.60479,10.66068,11.70900,12.36220,11.23974,6.19608,3
1997,10.92068,14.77332,17.78274,16.80466,14.89426,18.94097,18.77321,14.02466,18.37362,18.73837,15.64931,18.82848,20.00035,17.94840,18.87108,20.51561,18.89253,19.76897,22.51070,21.78256,18.06504,21.47733,21.21713,17.66917,21.06152,21.96842,19.87912,21.38633,22.51930,20.75471,20.82310,22.18846,20.41466,14.59780,18.50885,18.24570,14.43782,15.96470,16.07709,12.66522,...,9.18253,7.67860,7.07786,6.67404,8.93878,6.48787,5.32805,8.35738,4.68821,7.42377,7.77267,8.86730,8.05076,6.42138,8.27332,7.60790,6.96143,8.11448,5.75407,6.12906,8.32864,7.85715,8.58706,7.04518,8.67489,8.59362,8.34712,3.13330,8.58299,8.10538,8.12399,9.02117,8.84519,9.12127,4.67072,5.51102,7.80534,6.09219,7.00127,1
3075,8.98703,13.10976,15.52341,14.22476,13.06409,16.05604,15.29278,12.85476,16.33243,16.13428,14.03055,17.22843,17.57560,15.27046,18.51747,19.31997,16.84398,19.55072,20.96279,19.03682,18.85136,20.67201,19.45487,16.23510,18.28701,17.29348,16.05572,18.26759,17.80284,16.60596,19.48907,19.68331,17.21035,19.59653,20.51739,18.34393,18.26774,19.03046,17.42425,14.35214,...,6.68603,5.22903,6.92967,5.96869,7.25290,6.12962,6.13526,7.89154,7.10077,6.21776,6.69652,3.67499,2.20693,7.34579,5.80553,4.46963,5.17561,7.23040,6.80924,5.30671,4.78209,4.09962,6.64799,6.21480,6.08636,6.93228,4.54297,7.33285,6.50904,7.10691,5.38889,6.70536,6.77037,4.89895,4.46169,5.14842,5.62566,5.75016,5.81734,0
2174,9.93864,10.19106,14.75585,15.39269,12.27117,12.49056,16.01624,15.85107,12.45666,12.21919,13.10959,10.22746,7.72502,11.61733,12.51493,10.72227,8.63496,10.85512,10.79489,9.66945,7.37320,8.25052,7.13297,5.24253,7.77153,7.86910,7.55021,7.00340,8.11392,6.80905,5.77403,2.50073,6.43076,4.90474,4.69616,6.03373,6.65531,7.26686,4.66876,6.65406,...,6.34541,6.10667,5.17869,5.97196,6.89340,6.47448,4.46573,5.82089,5.57698,5.58734,5.23746,5.64929,6.11767,5.48836,5.60957,6.52082,4.41510,5.65138,5.67465,5.97785,6.02272,4.14522,6.50160,7.43982,6.39075,6.39053,6.92957,5.89852,5.17866,4.24545,3.67037,6.68640,6.38898,2.21925,5.69478,4.84930,4.25885,4.20736,5.04741,2
4308,9.83393,11.35302,15.63887,17.57798,16.51518,11.27298,18.05494,20.67247,19.72048,13.34157,15.74994,18.87376,18.36427,13.27917,12.12633,16.76762,16.55403,12.96372,6.69680,15.04791,15.21353,12.36276,8.78346,13.87378,14.81093,11.51154,9.70428,12.86143,13.91490,11.45130,8.74812,11.53898,12.66441,11.25270,7.48459,9.38136,12.33624,10.45300,9.61307,8.28866,...,8.30265,10.75524,9.92774,9.74084,10.70479,11.19898,10.76301,10.72790,9.92024,10.46769,9.62916,6.79223,9.25030,10.09292,9.35369,8.91991,9.35141,9.00810,8.83978,5.46741,8.96305,9.46246,9.23171,8.45667,9.99352,9.17458,9.84887,7.58047,10.08008,10.43447,9.01462,9.72251,11.17588,11.07127,10.74602,9.69299,12.04197,11.89917,7.90316,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

Split data into test and train set

In [0]:
test_set = df[:test_size]
train_set = df[test_size:]
assert len(test_set) + len(train_set) == len(df)

y_train = train_set['label'].values
x_train = train_set.drop('label', 1)
x_train = x_train.values
assert len(y_train) == len(x_train) 

y_test = test_set['label'].values
x_test = test_set.drop('label', 1)
x_test = x_test.values
assert len(y_test) == len(x_test) 

Compute mean of features for each class

In [24]:
means = []
for label in range(0,5):
  means.append(np.mean(x_train[y_train==label], axis=0))
# print(means)
print(np.shape(means))

(5, 256)


Compute Covarance matrix for each class

In [25]:
covs = []
for label in range(0,5):
  covs.append(EmpiricalCovariance().fit(x_train[y_train==label]).covariance_)
# print(means)
print(np.shape(covs))

(5, 256, 256)


Compute p for each class

In [26]:
p = []
for label in range(0,5):
  n = len(x_train[y_train==label])
  p.append(n/len(x_train))

print(np.shape(p))
print(p)

(5,)
[0.15548780487804878, 0.2292128603104213, 0.16352549889135254, 0.25914634146341464, 0.19262749445676275]


Compute Pdf 

In [28]:
y = []
for i in range(0,5):
  y.append(multivariate_normal.pdf(x_test, mean=means[i], cov=covs[i]) * p[i])
print(np.shape(y))

(5, 901)


Predict on test set

In [29]:
preds = np.argmax(y, axis=0)

print(np.shape(preds))

(901,)


Compute accuracy

In [30]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

0.8479467258601554


Reduce features with LDA

In [0]:
def compute_acc_lda(n_comps):

  lda = LDA(n_components=n_comps)
  X_train = lda.fit_transform(x_train, y_train)
  X_test = lda.transform(x_test)

  means = []
  for label in range(0,5):
    means.append(np.mean(X_train[y_train==label], axis=0))

  covs = []
  for label in range(0,5):
    covs.append(EmpiricalCovariance().fit(X_train[y_train==label]).covariance_)


  p = []
  for label in range(0,5):
    n = len(X_train[y_train==label])
    p.append(n/len(X_train))

  y = []
  for i in range(0,5):
    y.append(multivariate_normal.pdf(X_test, mean=means[i], cov=covs[i]) * p[i])

  preds = np.argmax(y, axis=0)

  return accuracy_score(y_test, preds)

In [22]:
for i in [4,3,2]:
  print(compute_acc_lda(i), 'accuracy score with components = ' + str(i) )

0.9200887902330743 accuracy score with components = 4
0.8790233074361821 accuracy score with components = 3
0.7369589345172031 accuracy score with components = 2
